# Roast My Resume Agent — Live Demo

**This is the presenter's demo notebook. Not for students.**

Watch the agent:
1. Analyze a resume bullet
2. Roast it brutally
3. Call a tool for recruiter context
4. Rewrite it
5. Argue with itself to find flaws
6. Produce the final, polished version

In [ ]:
# ============================================================
# SETUP — Run this first (one time)
# ============================================================
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata
import json
import time
from IPython.display import display, Markdown, HTML

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
model = genai.GenerativeModel('gemini-1.5-flash')

print("\u2705 Setup complete. Agent is online.")

In [ ]:
# ============================================================
# HELPER FUNCTIONS
# ============================================================

def stage_print(stage, icon, color, message):
    """Print a visually distinct stage header."""
    colors = {
        'cyan': '\033[96m',
        'red': '\033[91m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'magenta': '\033[95m',
        'white': '\033[97m',
        'reset': '\033[0m'
    }
    c = colors.get(color, colors['white'])
    r = colors['reset']
    print(f"\n{c}{'='*60}{r}")
    print(f"{c}{icon}  [ {stage} ]{r}")
    print(f"{c}{'='*60}{r}")
    time.sleep(0.5)
    print(f"\n{message}\n")


def thinking_dots(text, duration=1.5):
    """Simulate agent thinking."""
    import sys
    sys.stdout.write(f"\033[90m{text}")
    for _ in range(3):
        time.sleep(duration / 3)
        sys.stdout.write(".")
        sys.stdout.flush()
    print("\033[0m")


def get_role_requirements(role: str) -> str:
    """TOOL: Returns what recruiters look for in a specific role."""
    requirements = {
        "software engineer": "Strong DSA fundamentals, system design, clean code practices, CI/CD experience, testing frameworks, specific language/framework expertise (not just 'Python'), quantified impact metrics (%, $, users), open source contributions",
        "data scientist": "Statistical modeling, Python/R proficiency, SQL mastery, ML frameworks (scikit-learn, PyTorch), A/B testing design, business impact quantification, data pipeline experience, visualization skills",
        "product manager": "User research methodology, metrics-driven decisions (DAU, retention, conversion), roadmap planning, stakeholder management, market analysis, A/B testing, PRD writing",
        "ml engineer": "Model training and fine-tuning, MLOps (MLflow, Kubeflow), distributed training, model serving (TensorRT, ONNX), data pipeline engineering, experiment tracking, production deployment",
        "frontend developer": "React/Vue/Angular expertise, performance optimization (Core Web Vitals), accessibility (WCAG), responsive design, state management, testing (Jest, Cypress), design system experience",
        "backend developer": "API design (REST/GraphQL), database optimization (SQL + NoSQL), caching strategies, message queues, microservices architecture, monitoring/observability, load testing",
        "devops engineer": "CI/CD pipeline design, IaC (Terraform, Pulumi), container orchestration (K8s), monitoring (Prometheus, Grafana), cloud platforms (AWS/GCP/Azure), security best practices, incident response"
    }
    role_lower = role.lower()
    for key, value in requirements.items():
        if key in role_lower:
            return value
    return "Quantified achievements, specific technologies, measurable impact, action verbs, industry-specific keywords, problem-solving examples"


def call_llm(prompt):
    """Call Gemini and return parsed JSON."""
    response = model.generate_content(prompt)
    text = response.text.strip()
    # Strip markdown code fences if present
    if text.startswith('```'):
        text = text.split('\n', 1)[1]
    if text.endswith('```'):
        text = text.rsplit('```', 1)[0]
    return json.loads(text.strip())


print("\u2705 Helpers loaded.")

In [ ]:
# ============================================================
# THE SHOW — Run this cell during the live demo
# ============================================================

# --- CHANGE THESE FOR THE DEMO ---
resume_bullet = "Proficient in MS Office and team management"
target_role = "Software Engineer"
# ---------------------------------

print("\n" + "\033[97m" + "="*60 + "\033[0m")
print("\033[97m   ROAST MY RESUME AGENT — ACTIVATED\033[0m")
print("\033[97m" + "="*60 + "\033[0m")
print(f"\n\033[90mInput: \"{resume_bullet}\"\033[0m")
print(f"\033[90mTarget Role: {target_role}\033[0m")
time.sleep(1)

# --- STAGE 1: ANALYZING ---
stage_print("ANALYZING", "\ud83d\udd0d", "cyan", "Reading resume bullet and preparing initial analysis...")
thinking_dots("Processing")

analysis = call_llm(f"""You are a brutally honest resume reviewer. Analyze this resume bullet point.

Resume bullet: "{resume_bullet}"
Target role: {target_role}

Return ONLY valid JSON with:
- "score": integer 1-10 rating
- "first_impression": one sentence gut reaction
- "keywords_missing": list of 3-5 strong keywords that should be added
""")

print(f"  Score: {analysis['score']}/10")
print(f"  First Impression: {analysis['first_impression']}")
print(f"  Missing Keywords: {', '.join(analysis['keywords_missing'])}")
time.sleep(1.5)

# --- STAGE 2: ROASTING ---
stage_print("ROASTING", "\ud83d\udd25", "red", "Generating brutal critique...")
thinking_dots("Preparing roast")

roast = call_llm(f"""You are the Simon Cowell of resume reviews. Roast this resume bullet point.
Be funny but insightful. The person wants to improve, not cry.

Resume bullet: "{resume_bullet}"
Target role: {target_role}
Initial score: {analysis['score']}/10

Return ONLY valid JSON with:
- "roast": 2-3 sentences of brutal but constructive critique (be funny)
- "what_recruiter_thinks": one sentence from a recruiter's perspective
""")

print(f"  \ud83d\udca5 {roast['roast']}")
print(f"\n  \ud83d\udcbc Recruiter's Inner Monologue: \"{roast['what_recruiter_thinks']}\"")
time.sleep(1.5)

# --- STAGE 3: TOOL CALL ---
stage_print("TOOL CALL", "\ud83d\udee0\ufe0f", "yellow", f"Calling get_role_requirements('{target_role}')...")
thinking_dots("Fetching recruiter data")

requirements = get_role_requirements(target_role)
print(f"  Tool returned: {requirements}")
time.sleep(1.5)

# --- STAGE 4: REWRITING ---
stage_print("REWRITING", "\u270d\ufe0f", "green", "Generating improved version using recruiter context...")
thinking_dots("Crafting rewrite")

rewrite = call_llm(f"""You are a senior resume consultant. Rewrite this resume bullet for a {target_role} role.

Original: "{resume_bullet}"
Recruiter requirements for {target_role}: {requirements}
Missing keywords: {', '.join(analysis['keywords_missing'])}

Return ONLY valid JSON with:
- "rewrite": the improved bullet point (one concise sentence, starts with an action verb, includes metrics)
- "improvements_made": list of 3 specific changes you made
""")

print(f"  \u2728 {rewrite['rewrite']}")
print(f"\n  Changes made:")
for imp in rewrite['improvements_made']:
    print(f"    \u2713 {imp}")
time.sleep(1.5)

# --- STAGE 5: SELF-CRITIQUE ---
stage_print("SELF-CRITIQUE", "\ud83e\uddd0", "magenta", "Agent reviewing its own rewrite for flaws...")
thinking_dots("Self-reflecting")

critique = call_llm(f"""You are a hiring manager at a FAANG company. A junior resume writer produced this bullet:

"{rewrite['rewrite']}"

This is for a {target_role} role. Recruiters expect: {requirements}

Find exactly 3 specific problems with the rewrite and produce the FINAL, perfected version.

Return ONLY valid JSON with:
- "problems": list of exactly 3 specific issues
- "final_version": the ultimate perfected bullet point
- "ats_score": integer 1-100 ATS compatibility score for the final version
""")

print(f"  Issues found:")
for i, problem in enumerate(critique['problems'], 1):
    print(f"    {i}. {problem}")
time.sleep(1)

# --- STAGE 6: FINAL OUTPUT ---
stage_print("FINAL OUTPUT", "\u2705", "green", "")

print(f"\033[91m  BEFORE: \"{resume_bullet}\"\033[0m")
print(f"\033[90m  Score: {analysis['score']}/10\033[0m")
print()
print(f"\033[92m  AFTER:  \"{critique['final_version']}\"\033[0m")
print(f"\033[90m  ATS Score: {critique['ats_score']}/100\033[0m")
print()
print("\033[97m" + "="*60 + "\033[0m")
print("\033[97m   AGENT COMPLETE — 3 Agentic Patterns Used\033[0m")
print("\033[97m   Structured Output \u2192 Tool Calling \u2192 Self-Correction\033[0m")
print("\033[97m" + "="*60 + "\033[0m")

---

## Try Another One Live

Ask a volunteer for their resume bullet, paste it below, and run:

In [ ]:
# --- VOLUNTEER'S RESUME BULLET ---
resume_bullet = "Good communication skills and team player"
target_role = "Software Engineer"

# Copy the full pipeline from above and run it again
# (Or just re-run the cell above after changing the variables)